In [ ]:
# 라이브러리 Import Cell
import time
import pandas as pd
from tqdm import tqdm
from requests import request

tqdm.pandas()

# 리버스 엔지니어링
<br><br>
### 매장 URL
: https://www.starbucks.co.kr/store/getStore.do
- 조건에 맞는 매장 리스트를 요청하고, 응답으로 매장 리스트를 반환합니다.
- p_sido_cd의 값에 해당하는 시도의 전체 매장을 반환합니다.
<br><br>

### 시도 카테고리 URL
: https://www.starbucks.co.kr/store/getSidoList.do
- 시도 카테고리 목록을 반환합니다.
<br><br>

In [ ]:
# 상수 Cell

# 스크래핑인 것을 숨기기 위한 1차 가림막으로 headers 선언.
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
}

# 각 URI의 식별자를 Key로 지정.
# request 메서드와 URI로 이루어진 튜플을 Value로 지정.
URI = {
    'STORE_LIST': ('post', 'https://www.starbucks.co.kr/store/getStore.do'),
    'CAT_SIDO': ('post', 'https://www.starbucks.co.kr/store/getSidoList.do'),
    'CAT_GUGUN': ('post', 'https://www.starbucks.co.kr/store/getGugunList.do')
}

In [ ]:
# 변수 Cell

# 각 요청에 필요한 데이터 저장
PAYLOAD = {
    'STORE_LIST': {
        'in_biz_cds': '0',
        'in_scodes': '0',
        'ins_lat': '37.56682',
        'ins_lng': '126.97865',
        'search_text': '',
        'p_sido_cd': '01',
        'p_gugun_cd': '',
        'isError': 'true',
        'in_distance': '0',
        'in_biz_cd': '',
        'iend': '1000',
        'searchType': 'C',
        'set_date': '',
        'todayPop': '0',
        'all_store': '0',
        'T03': '0',
        'T01': '0',
        'T27': '0',
        'T12': '0',
        'T09': '0',
        'T30': '0',
        'T05': '0',
        'T22': '0',
        'T21': '0',
        'T10': '0',
        'T36': '0',
        'T43': '0',
        'T48': '0',
        'P10': '0',
        'P50': '0',
        'P20': '0',
        'P60': '0',
        'P30': '0',
        'P70': '0',
        'P40': '0',
        'P80': '0',
        'whcroad_yn': '0',
        'P90': '0',
        'new_bool': '0'
    },
    'CAT_SIDO': {},
    'CAT_GUGUN': {}
}

COLS = {
    'STORE': {
        'sido_code': '시도 코드',
        'sido_name': '시도 명',
        'gugun_code': '구군 코드',
        'gugun_name': '구군 명',
        'addr': '주소',
        'doro_address': '도로명 주소',
        'tel': '전화 번호',
        'fax': '팩스 번호',
        'my_siren_order_store_yn': '나의 사이렌 오더 여부',
        'lat': '위도',
        'lot': '경도'
    },
    'CAT_SIDO': {
        'sido_cd': '시도 코드',
        'sido_nm': '시도 명'
    }
    
}

In [ ]:
# pandas.read_json() 으로 요청 시도.
pd.read_json(URL['CAT_SIDO'][1])

# status 404: 요청한 페이지가 존재하지 않음. 
# pandas.read_json() 은 POST를 지원하지 않는 것으로 보여짐.

In [ ]:
# requests post 로 시도 카테고리 가져오기
resp_cat_sido = request(*URI['CAT_SIDO'], headers=HEADERS)
resp_cat_sido

In [ ]:
# json 데이터로 변환
json_cat_sido = resp_cat_sido.json()
json_cat_sido

In [ ]:
# 데이터프레임으로 변환
df_cat_sido = pd.DataFrame(json_cat_sido['list'])
df_cat_sido

In [ ]:
# 데이터프레임 전처리
df_cat_sido = df_cat_sido[['sido_cd', 'sido_nm']]
df_cat_sido

In [ ]:
# sido 데이터를 이용해 구군을 선택하지 않고 시도의 전체 매장 불러오기 시도
resp_store_list = request(*URI['STORE_LIST'], headers=HEADERS, data=PAYLOAD['STORE_LIST'])
resp_store_list

In [ ]:
# 서울 시의 599개의 매장 정보 가져오기 성공, json 변환
json_store_list = resp_store_list.json()
json_store_list

In [ ]:
# json 데이터를 데이터프레임으로 변환
df_store_list = pd.DataFrame(json_store_list['list'])
df_store_list

In [ ]:
# 데이터프레임 전처리

# 컬럼 순서 변경
df_store_list = df_store_list[list(COLS['STORE'].keys())]

# 컬럼 번역
df_store_list.columns = list(COLS['STORE'].values())

# 위도, 경도 형 변환
df_store_list = df_store_list.astype({'위도': float, '경도': float})

df_store_list

In [ ]:
# 데이터프레임 정보 확인 및 결측치 여부 확인
df_store_list.info()

# 함수화
<br>

### 시도 카테고리 수집 함수
1. 시도 카테고리 정보를 요청한다.
2. 받은 응답값을 전처리하여 반환한다.
<br><br>

### 시도의 전체 매장 수집 함수
1. 시도 카테고리를 입력받는다.
2. 시도의 전체 매장 정보를 요청한다.
3. 받은 응답값을 전처리한다.
4. 전처리된 데이터프레임을 반환한다.
<br><br>

In [ ]:
# 시도 카테고리 수집 함수
def get_cat_sido():
    """
    시도 카테고리 수집 함수.
    
    시도 카테고리 API에 요청을 보낸 후, 응답을 받습니다.
    받은 응답은 response => json => DataFrame 순으로 형 변환 과정을 거칩니다.
    형 변환이 모두 완료되면, 사용할 컬럼만 가져옵니다.
    컬럼명을 한국어로 번역하여 반환합니다.
    
    -------
    :return pandas.DataFrame: 수집된 '시도 코드', '시도 명' DataFrame
    """
    
    # 시도 카테고리 API 요청 및 response 객체 초기화
    resp_cat_sido = request(*URI['CAT_SIDO'], headers=HEADERS)
    
    # response 객체 => json 형 변환
    json_cat_sido = resp_cat_sido.json()
    
    # json => DataFrame 형 변환
    df_cat_sido = pd.DataFrame(json_cat_sido['list'])
    
    # 컬럼 정리
    df_cat_sido = df_cat_sido[list(COLS['CAT_SIDO'].keys())]
    
    # 컬럼 번역
    df_cat_sido.columns = list(COLS['CAT_SIDO'].values())
    
    # DataFrame 반환
    return df_cat_sido


get_cat_sido()

In [ ]:
# 시도의 전체 매장 수집 함수
def get_sido_store(sido_code: str):
    """
    시도 전체 매장 수집 함수.
    
    매장 정보 API에 요청을 보낸 후, 응답을 받습니다.
    받은 응답은 response => json => DataFrame 순으로 형 변환 과정을 거칩니다.
    형 변환이 모두 완료되면, 사용할 컬럼만 가져온 후 컬럼명을 한국어로 번역합니다.
    object로 선언되어 있는 위도, 경도 데이터를 float으로 형 변환한 후 반환합니다.
    
    ------
    params sido_code: str, 시도 코드
    
    -------
    :return pandas.DataFrame: 수집된 매장 정보 DataFrame
    """
    
    # 입력받은 시도 코드 파라미터로 Request Data 초기화
    PAYLOAD['STORE_LIST']['p_sido_cd'] = sido_code
    
    # 시도 카테고리 API 요청 및 response 객체 초기화
    resp_store_list = request(*URI['STORE_LIST'], headers=HEADERS, data=PAYLOAD['STORE_LIST'])
    
    # response 객체 => json 형 변환
    json_store_list = resp_store_list.json()
    
    # json => DataFrame 형 변환
    df_store_list = pd.DataFrame(json_store_list['list'])
    
    # 컬럼 정리
    df_store_list = df_store_list[list(COLS['STORE'].keys())]
    
    # 컬럼 번역
    df_store_list.columns = list(COLS['STORE'].values())
    
    # object로 선언된 위도, 경도 데이터 float로 형 변환
    df_store_list = df_store_list.astype({'위도': float, '경도': float})
    
    # DataFrame 반환
    return df_store_list
    
    
get_sido_store('01')

In [ ]:
df_store = pd.concat(list(get_cat_sido()['시도 코드'].progress_map(get_sido_store)), ignore_index=True)

In [ ]:
df_store

In [ ]:
df_store.to_csv('./data/starbucks/starbucks_stores.csv', index=False)

In [ ]:
pd.read_csv('./data/starbucks/starbucks_stores.csv', dtype={'시도 코드': object, '구군 코드': object})